In [ ]:
from dataclasses import dataclass
from datetime import datetime
import os
import numpy as np 

DATABASE_NAME = 'NETWORK_SECURITY'
COLLECTION_NAME = 'NETWORK_DATA' 
MONGODB_URL = 'MONGODB_URL'

ARTIFACTS_DIR = 'artifacts'
PIPELINE_DIR = 'security'

DATA_INGESTION_DIR_NAME : str = 'data_ingestion'
DATA_INGESTION_COLLECTION_NAME: str = 'NETWORK_DATA'
DATA_INGESTION_FEATURE_STORED_NAME:str = 'feature'
DATA_INGESTION_INGESTED_NAME:str = 'ingested'
DATA_INGESTION_SPLIT_RATIO:float = 0.2 

RAW_DATA = 'security.csv'
TRAIN_DATA = 'train.csv'
TEST_DATA = 'test.csv'

TIMESTAMP = datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

@dataclass 
class TrainingConfiguration:
    artifact_dir:str = ARTIFACTS_DIR 
    piprline_dir:str = PIPELINE_DIR 
    timestamp:str = TIMESTAMP 

trainingconfig : TrainingConfiguration=TrainingConfiguration()

@dataclass 
class Data_ingestion_configeration:
    data_ingestion_dir:str = os.path.join(trainingconfig.artifact_dir,DATA_INGESTION_DIR_NAME)
    data_ingestion_collection:str = DATA_INGESTION_COLLECTION_NAME 
    data_ingestion_feature:str = os.path.join(data_ingestion_dir,DATA_INGESTION_FEATURE_STORED_NAME,RAW_DATA)
    train_data_path:str = os.path.join(data_ingestion_dir,DATA_INGESTION_INGESTED_NAME,TRAIN_DATA)
    test_data_path:str = os.path.join(data_ingestion_dir,DATA_INGESTION_INGESTED_NAME,TEST_DATA)
    split_ratio:float = DATA_INGESTION_SPLIT_RATIO


import pymongo 
import pandas as pd 
import certifi 
import sys


ca = certifi()
class MongoDBClient:
    try:
        def __init__(self,dt_base=DATABASE_NAME):
            if MongoDBClient.client is None:
                mongo_url = os.getenv(MONGODB_URL)
                if mongo_url is None:
                    raise Exception (f"ERROR:-> Connection Failed")
            MongoDBClient.Client = pymongo.MongoClient(mongo_url,tlsCAFiles= ca)
            self.Client=MongoDBClient.Client
            self.database = self.Client[dt_base]
            self.database_name = dt_base
    except Exception as e:
        raise (e,sys)
        

from typing import Optional
from sklearn.model_selection import train_test_split

class Network_data_collection_to_dataframe:
    try:
        def __init__(self):
            self.mongodbclient = MongoDBClient()
        def get_data(self,connection_name:str,database_name:Optional[str]=None)-> pd.DataFrame:
            if database_name is None:
                connection = self.mongodbclient.database[connection_name]
            else:
                connection = self.mongodbclient[database_name][connection_name]
            df = pd.DataFrame(list(connection.find()))
            if '_id' in df.columns.to_list():
                df = df.drop('_id',axis=1)
            df.replace({'na',np.nan},inplace=True)
            return df
    except Exception as e:
        raise (e,sys)

@dataclass 
class TrainingPipelineConfig:
    TRAIN_DATA_PATH:str 
    TEST_DATA:str 

class Data_Ingestion:
    def __init__(self,data_ingestion_config=Data_ingestion_configeration):
        self.data_ingestion_config = data_ingestion_config

    def extract_to_feature_data(self):
        network = Network_data_collection_to_dataframe()
        network_data = network.get_data(self.data_ingestion_config.data_ingestion_collection)
        feature_stored_path = self.data_ingestion_config.data_ingestion_feature
        
        os.makedirs(os.path.dirname(feature_stored_path),exist_ok=True)
        network_data.to_csv(feature_stored_path,index=False,header=True)
        return network_data
    
    def split_data(self,dataframe):
        train,test = train_test_split(dataframe,test_size=self.data_ingestion_config.split_ratio)
        # Train save
        train_file_path = self.data_ingestion_config.train_data_path
        os.makedirs(os.path.dirname(train_file_path), exist_ok=True)
        train.to_csv(train_file_path, index=False, header=True)

        # Test save
        test_file_path = self.data_ingestion_config.test_data_path
        os.makedirs(os.path.dirname(test_file_path), exist_ok=True)
        test.to_csv(test_file_path, index=False, header=True)
        

    def init_data_ingetion(self):
        dataframe = self.extract_to_feature_data()
        self.split_data(dataframe)

        data_pipeline_training = TrainingPipelineConfig(
            self.data_ingestion_config.train_data_path,
            self.data_ingestion_config.test_data_path,
        )
        return data_pipeline_training




# data_ingestion_config = DataIngestionConfiguration()

# data_ingestion = DataIngestion(data_ingestion_config)
# pipeline_config = data_ingestion.init_data_ingestion()

# print("Train Data Path:", pipeline_config.train_data_path)
# print("Test Data Path:", pipeline_config.test_data_path)


In [2]:
! pip install scikit-learn

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   - -----------------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\tanvi\\AppData\\Local\\Temp\\pip-unpack-bhol1fv8\\scipy-1.16.2-cp312-cp312-win_amd64.whl'
Consider using the `--user` option or check the permissions.

